In [ ]:
!pip install pandas networkx matplotlib spacy scipy
!pip install -U spacy
!python -m spacy download pt_core_news_lg
!pip install liac-arff
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB ? eta 0:00:000:00:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11768 sha256=2a8b548353c3e4987e01727376a81d6406aa33041f790d2d3ee15385e9016aee
  Stored in directory: /home/loli/.cache/pip/wheels/a9/ac/cf/c2919807a5c623926d217c0a18eb5b457e5c19d242c3b5963a
Successfully built liac-arff


In [1]:
import pandas as pd              # Para manipular os dados (CSVs, DataFrames)
import re                        # Para usar expressões regulares na limpeza do texto (remover @)
import networkx as nx            # Para criar e manipular os grafos
import matplotlib.pyplot as plt  # Para visualizar os grafos
import spacy                     # A biblioteca principal para processamento de linguagem em português
from itertools import combinations
import spacy
import arff
import os

In [2]:
# Carregar o modelo de linguagem em português do spaCy
# Isso substitui os downloads do NLTK
try:
    nlp = spacy.load('pt_core_news_lg')
    print("Bibliotecas essenciais e modelo 'pt_core_news_lg' do spaCy carregados com sucesso!")
except OSError:
    print("Modelo 'pt_core_news_sm' não encontrado. Por favor, execute a célula de instalação:")
    print("!python -m spacy download pt_core_news_lg")

Bibliotecas essenciais e modelo 'pt_core_news_lg' do spaCy carregados com sucesso!


In [3]:
offComBR_01 = './data/01-raw/offComBR/OffComBR3.arff'
# Carrega o arquivo .arff
# Carrega o ARFF com liac-arff
with open(offComBR_01, 'r', encoding='utf-8') as f:
    arff_data = arff.load(f)

# Converte para DataFrame
df_arff = pd.DataFrame(arff_data['data'], columns=[attr[0] for attr in arff_data['attributes']])

print("--- Pré-visualização dos dados do .arff ---")
print(df_arff.head())

caminho_arff_saida = './data/01-raw/offComBR/offComBR.csv'
# Salva o DataFrame como .csvcaminho_csv_saida_1
df_arff.to_csv('./data/01-raw/offComBR/offComBR.csv', index=False, encoding='utf-8')

print(f"\n[SUCESSO] Arquivo '{offComBR_01}' convertido para '{caminho_arff_saida}'")

--- Pré-visualização dos dados do .arff ---
  @@class                                           document
0     yes               Votaram no PEZAO Agora tomem no CZAO
1      no  cuidado com a poupanca pessoal Lembram o que a...
2      no  Sabe o que eu acho engracado os nossos governa...
3      no              Podiam retirar dos lucros dos bancos 
4      no  CADE O GALVAO PRA NARRAR AGORA   FALIIIIUUUUUU...

[SUCESSO] Arquivo './data/01-raw/offComBR/OffComBR3.arff' convertido para './data/01-raw/offComBR/offComBR.csv'


In [7]:
olidBR_01 = './data/01-raw/olidBR/test-00000-of-00001-914dbee7561d2266.parquet'
df_parquet = pd.read_parquet(olidBR_01)
print("--- Pré-visualização dos dados do .parquet ---")
print(df_parquet.head())

caminho_parquet_saida = './data/01-raw/olidBR/olidBR.csv'
# Salva o DataFrame como .csv
df_parquet.to_csv(caminho_parquet_saida, index=False, encoding='utf-8')

print(f"\n[SUCESSO] Arquivo '{olidBR_01}' convertido para '{caminho_parquet_saida}'")

--- Pré-visualização dos dados do .parquet ---
                                 id  \
0  da19df36730945f08df3d09efa354876   
1  80f1a8c981864887b13963fed1261acc   
2  80eee9db811c4ea4b2ddb7863d12c5fe   
3  2f67025f913e4a6292e3d000d9e2b5a8   
4  e64148caa4474fc79298e01d0dda8f5e   

                                                text is_offensive is_targeted  \
0  USER Adorei o comercial também Jesus. Só achei...          OFF         UNT   
1  Cara isso foi muito babaca geral USER conhece ...          OFF         TIN   
2                           Quem liga pra judeu kkkk          OFF         UNT   
3  Se vc for porco, folgado e relaxado, você não ...          OFF         UNT   
4    USER Toma no cu é vitamina como tu e tua prima.          OFF         TIN   

  targeted_type                                        toxic_spans  health  \
0          None  [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6...   False   
1           GRP  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   False 

In [21]:
datasets_01 = {
    "hateBR": './data/01-raw/hateBR/HateBRXplain.csv',
    "offComBR": './data/01-raw/offComBR/offComBR.csv',
    "olidBR": './data/01-raw/olidBR/olidBR.csv',
    "fortuna": './data/01-raw/fortuna/2019-05-28_portuguese_hate_speech_binary_classification.csv',
    "toLDBR": './data/01-raw/toLDBR/ToLD-BR.csv',
    "tuPyE": './data/01-raw/tuPyE/binary_test.csv'
}

# Lê e conta as instâncias
print("--- Total de instâncias por dataset ---")
for nome, caminho in datasets_01.items():
    try:
        df = pd.read_csv(caminho)
        print(f"\n{nome} ({df.shape[0]} instâncias):")
        print(list(df.columns))
    except Exception as e:
        print(f"{nome}: Erro ao ler o arquivo - {e}")

--- Total de instâncias por dataset ---

hateBR (7000 instâncias):
['id', 'comment', 'offensive_label', 'link_post', 'rationales_annotator1', 'rationales_annotator2']

offComBR (1033 instâncias):
['class', 'document']

olidBR (1738 instâncias):
['id', 'text', 'is_offensive', 'is_targeted', 'targeted_type', 'toxic_spans', 'health', 'ideology', 'insult', 'lgbtqphobia', 'other_lifestyle', 'physical_aspects', 'profanity_obscene', 'racism', 'religious_intolerance', 'sexism', 'xenophobia']

fortuna (5670 instâncias):
['text', 'hatespeech_comb', 'hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3']

toLDBR (21000 instâncias):
['text', 'homophobia', 'obscene', 'insult', 'racism', 'misogyny', 'xenophobia']

tuPyE (8734 instâncias):
['source', 'text', 'researcher', 'year', 'aggressive', 'hate']


In [ ]:
def cont_categorias(coluna_alvo, nome, value):
    df = pd.read_csv(datasets_01[nome])
    total = (df[coluna_alvo] == value).sum()
    print(f"{nome}: {total} instâncias na {coluna_alvo}")

cont_categorias('offensive_label','hateBR', 1)
cont_categorias('is_offensive','olidBR', 'OFF')
cont_categorias('hatespeech_comb','fortuna', 1)

colunas_odio = ['homophobia', 'obscene', 'insult', 'racism', 'misogyny', 'xenophobia']
df = pd.read_csv(datasets_01['toLDBR'])
df['hate'] = df[colunas_odio].any(axis=1).astype(int)
df.to_csv('./data/01-raw/toLDBR/ToLD-BR.csv', index=False, encoding='utf-8')
cont_categorias('hate','toLDBR', 1)

cont_categorias('hate','tuPyE', 1)

hateBR: 3500 instâncias na offensive_label
olidBR: 1484 instâncias na is_offensive
fortuna: 1788 instâncias na hatespeech_comb
toLDBR: 9255 instâncias na hate
tuPyE: 1051 instâncias na hate


In [35]:
datasets_02 = {
     "hateBR": {
        'col_mensagem': 'comment',
        'col_rotulo': 'offensive_label',
        'saida': './data/02-cleaned/hateBR/HateBRXplain.csv'
    },
    "offComBR": {
        'col_mensagem': 'document',             
        'col_rotulo': 'class',
        'saida': './data/02-cleaned/offComBR/offComBR.csv'
    },
    "olidBR": {
        'col_mensagem': 'text',
        'col_rotulo': 'is_offensive',
        'saida': './data/02-cleaned/olidBR/olidBR.csv'
    },
    "fortuna": {
        'col_mensagem': 'text',
        'col_rotulo': 'hatespeech_comb',
        'saida': './data/02-cleaned/fortuna/fortuna.csv'
    },
    "toLDBR": {
        'col_mensagem': 'text',
        'col_rotulo': 'hate',
        'saida': './data/02-cleaned/toLDBR/ToLD-BR.csv'
    },
    "tuPyE": {
        'col_mensagem': 'text',
        'col_rotulo': 'hate',
        'saida': './data/02-cleaned/tuPyE/tuPyE.csv'
    }
}

In [36]:
def uniformizar_csv(nome, caminho, col_mensagem, col_rotulo, saida):
    try:
        df = pd.read_csv(caminho)

        # Verifica se as colunas estão no DataFrame
        if col_mensagem in df.columns and col_rotulo in df.columns:
            df_padronizado = df[[col_mensagem, col_rotulo]].rename(columns={
                col_mensagem: 'mensagem',
                col_rotulo: 'odio'
            })

            # Caminho de saída
            os.makedirs(os.path.dirname(saida), exist_ok=True)
            caminho_saida = os.path.join(saida, f"{nome}.csv")
            df_padronizado.to_csv(saida, index=False, encoding='utf-8')

            print(f"[✓] {nome}: {df_padronizado.shape[0]} instâncias salvas em '{caminho_saida}'")
        else:
            print(f"[✗] {nome}: colunas '{col_mensagem}' e/ou '{col_rotulo}' não encontradas.")

    except Exception as e:
        print(f"[ERRO] {nome}: {e}")
        
for nome, config in datasets_02.items():
    uniformizar_csv(
        nome,
        datasets_01[nome],
        config['col_mensagem'],
        config['col_rotulo'],
        config['saida']
    )

[✓] hateBR: 7000 instâncias salvas em './data/02-cleaned/hateBR/HateBRXplain.csv/hateBR.csv'
[✓] offComBR: 1033 instâncias salvas em './data/02-cleaned/offComBR/offComBR.csv/offComBR.csv'
[✓] olidBR: 1738 instâncias salvas em './data/02-cleaned/olidBR/olidBR.csv/olidBR.csv'
[✓] fortuna: 5670 instâncias salvas em './data/02-cleaned/fortuna/fortuna.csv/fortuna.csv'
[✓] toLDBR: 21000 instâncias salvas em './data/02-cleaned/toLDBR/ToLD-BR.csv/toLDBR.csv'
[✓] tuPyE: 8734 instâncias salvas em './data/02-cleaned/tuPyE/tuPyE.csv/tuPyE.csv'


In [ ]:
def limpar_e_lematizar(texto: str) -> str:
    """
    Limpa o texto e aplica lematização com spaCy para português.
    Remove menções, hashtags, emojis, stopwords e pontuações.
    """
    # Remover menções (@usuario)
    texto = re.sub(r'@\w+', '', texto)
    # Remover hashtags (#palavra) mas manter a palavra
    texto = re.sub(r'#', '', texto)
    # Remover emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas
        u"\U0001F680-\U0001F6FF"  # Transporte e mapas
        u"\U0001F1E0-\U0001F1FF"  # Bandeiras
        u"\U00002500-\U00002BEF"  # Caracteres diversos
        u"\U00002700-\U000027BF"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    texto = emoji_pattern.sub(r'', texto)

    # Converter para minúsculas
    texto = texto.lower()

    # Processar o texto com spaCy
    doc = nlp(texto)

    # Lematizar, ignorando pontuação, stopwords e espaços
    lemmas = [
        token.lemma_ for token in doc 
        if not token.is_punct and not token.is_space and not token.is_stop
    ]

    return " ".join(lemmas)

In [ ]:
def pre_processar_pt(caminho_csv: str, nome_coluna: str) -> pd.DataFrame:
    """
    Carrega um CSV, remove duplicatas e aplica limpeza e lematização otimizada
    para o português em uma coluna de texto.
    """
    # 1. Carregar o dataset
    df = pd.read_csv(caminho_csv)
    print(f"Dataset original carregado com {len(df)} linhas.")

    # 2. Remover duplicatas
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Dataset após remoção de duplicatas possui {len(df)} linhas.")

    # 3. Limpeza e Lematização com spaCy
    # Criar uma nova coluna para o texto processado
    df['texto_processado'] = df[nome_coluna].apply(lambda texto: limpar_e_lematizar_spacy(texto))

    return df

In [ ]:
# A função de montar o grafo permanece a mesma
def montar_grafo_de_coocorrencia(series_texto: pd.Series) -> nx.Graph:
    """
    Cria um grafo NetworkX a partir de uma série de textos processados.
    Nós = palavras (lemas)
    Arestas = coocorrência de palavras na mesma sentença (tweet/comentário)
    """
    G = nx.Graph()
    
    for texto in series_texto:
        palavras = texto.split()
        
        # Adiciona nós (evita duplicatas automaticamente)
        for palavra in palavras:
            G.add_node(palavra)
            
        # Adiciona arestas baseadas nas combinações de palavras no mesmo texto
        for w1, w2 in combinations(palavras, 2):
            if G.has_edge(w1, w2):
                if 'weight' in G[w1][w2]:
                    G[w1][w2]['weight'] += 1
                else:
                     G[w1][w2]['weight'] = 2
            else:
                G.add_edge(w1, w2, weight=1)
                
    return G